In [126]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq
import re
import nographs as nog

In [ ]:
def read_input(puzzle_input):
    with open(puzzle_input) as ff:
        dd = ff.readlines()

    valves = {}
    for ll in dd:
        mm = re.match("Valve (\w+) has flow rate=(\d+); (?:tunnels|tunnel) (?:leads|lead) to (?:valves|valve) (.*)", ll.strip())
        if mm:
            adj = [ xx.strip() for xx in mm.group(3).split(",") ]
            valves[mm.group(1)] = (int(mm.group(2)), adj)
    
    return valves

display(read_input(os.path.join(os.path.join("..", "dat", "day16_test.txt"))))

def part1(puzzle_input):
    valves = read_input(puzzle_input)

    def next_edges(state, t):
        minutes_left = 30 - t.depth -1

        if minutes_left == 1:
            return

        pos = state[0]
        accumflow = state[1]
        opened = set(state[2])

        if pos not in opened and valves[pos][0] > 0:
            # if the valve hasn't been opened and it's worth sticking around, open it
            _opened = set(state[2])
            _opened.add(pos)
            flow_if_opened = valves[pos][0] * minutes_left
            yield (pos, accumflow+flow_if_opened, tuple(_opened)), 1
        
        if len(opened) == len(valves):
            # if all valves have been opened we don't need to go anywhere
            return
        else:
            for adj in valves[pos][1]:
                yield (adj, state[1], tuple(opened)), 1

    traversal = nog.TraversalShortestPaths(next_edges)
    max_flow = (None, 0)
    for vertex in traversal.start_from(('AA',0,tuple(),30), build_paths=True):       
        #print(vertex, sum([ valves[xx][0] for xx in vertex[2] ]), traversal.depth)
        if vertex[1] > max_flow[1]:
            max_flow = vertex
    print("MAX", max_flow)
    for ii, path in enumerate(traversal.paths[max_flow]): 
        print(ii+1, path)

    return max_flow[1]

ans = part1(os.path.join(os.path.join("..", "dat", "day16_test.txt")))
assert ans == 1651

ans = part1(os.path.join(os.path.join("..", "dat", "day16.txt")))
print(ans)

{'AA': (0, ['DD', 'II', 'BB']),
 'BB': (13, ['CC', 'AA']),
 'CC': (2, ['DD', 'BB']),
 'DD': (20, ['CC', 'AA', 'EE']),
 'EE': (3, ['FF', 'DD']),
 'FF': (0, ['EE', 'GG']),
 'GG': (0, ['FF', 'HH']),
 'HH': (22, ['GG']),
 'II': (0, ['AA', 'JJ']),
 'JJ': (21, ['II'])}

MAX ('CC', 1651, ('JJ', 'BB', 'HH', 'DD', 'EE', 'CC'))
1 ('AA', 0, (), 30)
2 ('DD', 0, ())
3 ('DD', 560, ('DD',))
4 ('AA', 560, ('DD',))
5 ('BB', 560, ('DD',))
6 ('BB', 885, ('DD', 'BB'))
7 ('AA', 885, ('DD', 'BB'))
8 ('II', 885, ('DD', 'BB'))
9 ('JJ', 885, ('DD', 'BB'))
10 ('JJ', 1326, ('DD', 'JJ', 'BB'))
11 ('II', 1326, ('DD', 'JJ', 'BB'))
12 ('AA', 1326, ('DD', 'JJ', 'BB'))
13 ('DD', 1326, ('DD', 'JJ', 'BB'))
14 ('EE', 1326, ('DD', 'JJ', 'BB'))
15 ('FF', 1326, ('DD', 'JJ', 'BB'))
16 ('GG', 1326, ('DD', 'JJ', 'BB'))
17 ('HH', 1326, ('DD', 'JJ', 'BB'))
18 ('HH', 1612, ('DD', 'HH', 'JJ', 'BB'))
19 ('GG', 1612, ('DD', 'HH', 'JJ', 'BB'))
20 ('FF', 1612, ('DD', 'HH', 'JJ', 'BB'))
21 ('EE', 1612, ('DD', 'HH', 'JJ', 'BB'))
22 ('EE', 1639, ('JJ', 'BB', 'HH', 'DD', 'EE'))
23 ('DD', 1639, ('JJ', 'BB', 'HH', 'DD', 'EE'))
24 ('CC', 1639, ('JJ', 'BB', 'HH', 'DD', 'EE'))
25 ('CC', 1651, ('JJ', 'BB', 'HH', 'DD', 'EE', 'CC'))
MAX ('IQ', 1376, ('HO', 'TJ', 'HS', 'EG', 'IQ'))
1 ('AA', 0, (), 30)
2 ('V

In [139]:
def read_input(puzzle_input):
    with open(puzzle_input) as ff:
        dd = ff.readlines()

    valves = {}
    for ll in dd:
        mm = re.match("Valve (\w+) has flow rate=(\d+); (?:tunnels|tunnel) (?:leads|lead) to (?:valves|valve) (.*)", ll.strip())
        if mm:
            adj = [ xx.strip() for xx in mm.group(3).split(",") ]
            valves[mm.group(1)] = (int(mm.group(2)), adj)
    
    return valves

display(read_input(os.path.join(os.path.join("..", "dat", "day16_test.txt"))))

prune_cnt = 0

def part1(puzzle_input):
    global prune_cnt
    valves = read_input(puzzle_input)

    non_zero_valves = set()
    for valve, (flow, _) in valves.items():
        if flow > 0:
            non_zero_valves.add(valve)

    max_flows = [ 0 ] * 27
    
    def next_edges(state, t):
        global prune_cnt
        minutes_left = 26 - t.depth - 1

        if minutes_left == 0:
            return

        p_pos = state[0]
        e_pos = state[1]
        accumflow = state[2]
        opened = set(state[3])

        if len(opened) == len(non_zero_valves):
            # if all valves have been opened we don't need to go anywhere
            return

        valves_left_open = (non_zero_valves - opened)
        flows_left = [ valves[xx][0] for xx in valves_left_open]
        flows_left.sort(reverse=True)
        maximum_valves_to_open = (minutes_left//2) + 1
        bestflow = accumflow
        for ii, flow in enumerate(sorted(flows_left, reverse=True)):
            if ii < maximum_valves_to_open:
                bestflow += (flow*(minutes_left-(ii*2)))
            else:
                break

        if bestflow < max_flows[t.depth]:
            prune_cnt = prune_cnt + 1
            return

        if (p_pos != e_pos) and (p_pos not in opened and valves[p_pos][0] > 0) and (e_pos not in opened and valves[e_pos][0] > 0):
            # both elephant and person open valve
            _opened = set(state[3])
            _opened.add(p_pos)
            _opened.add(e_pos)
            flow_if_opened = (valves[p_pos][0] + valves[e_pos][0]) * minutes_left
            newflow = accumflow+flow_if_opened
            max_flows[t.depth+1] = max(max_flows[t.depth+1], newflow)
            yield (p_pos, e_pos, newflow, tuple(_opened)) #, 1

        if (p_pos not in opened and valves[p_pos][0] > 0):
            # person opens valve
            _opened = set(state[3])
            _opened.add(p_pos)
            flow_if_opened = valves[p_pos][0] * minutes_left
            newflow = accumflow+flow_if_opened
            max_flows[t.depth+1] = max(max_flows[t.depth+1], newflow)
            for adj in valves[e_pos][1]:
                yield (p_pos, adj, newflow, tuple(_opened)) #, 1

        if (e_pos not in opened and valves[e_pos][0] > 0):
            # elphant opens valve
            _opened = set(state[3])
            _opened.add(e_pos)
            flow_if_opened = valves[e_pos][0] * minutes_left
            newflow = accumflow+flow_if_opened
            max_flows[t.depth+1] = max(max_flows[t.depth+1], newflow)
            for adj in valves[p_pos][1]:
                yield (adj, e_pos, newflow, tuple(_opened)) #, 1
        
        # both move
        for p_adj in valves[p_pos][1]:
            for e_adj in valves[e_pos][1]:
                max_flows[t.depth+1] = max(max_flows[t.depth+1], state[2])
                yield (p_adj, e_adj, state[2], tuple(opened)) #, 1

    traversal = nog.TraversalBreadthFirst(next_edges)
    max_flow = (None, None, 0)
    last_depth = 0
    for vertex in traversal.start_from(('AA','AA',0,tuple(),30)):       
        #print(vertex, traversal.depth)
        if last_depth != traversal.depth:
            print("Depth", traversal.depth, prune_cnt)
            last_depth = traversal.depth
        if vertex[2] > max_flow[2]:
            max_flow = vertex
    print("MAX", max_flow)
    #for ii, path in enumerate(traversal.paths[max_flow]): 
    #    print(ii+1, path)

    return max_flow[2]

ans = part1(os.path.join(os.path.join("..", "dat", "day16_test.txt")))
assert ans == 1707

ans = part1(os.path.join(os.path.join("..", "dat", "day16.txt")))
print(ans) #1926 is too low

{'AA': (0, ['DD', 'II', 'BB']),
 'BB': (13, ['CC', 'AA']),
 'CC': (2, ['DD', 'BB']),
 'DD': (20, ['CC', 'AA', 'EE']),
 'EE': (3, ['FF', 'DD']),
 'FF': (0, ['EE', 'GG']),
 'GG': (0, ['FF', 'HH']),
 'HH': (22, ['GG']),
 'II': (0, ['AA', 'JJ']),
 'JJ': (21, ['II'])}

Depth 1 0
Depth 2 0
Depth 3 0
Depth 4 0
Depth 5 0
Depth 6 0
Depth 7 0
Depth 8 78
Depth 9 3265
Depth 10 3989
Depth 11 4189
MAX ('EE', 'CC', 1707, ('JJ', 'BB', 'HH', 'DD', 'EE', 'CC'))
Depth 1 4233
Depth 2 4233
Depth 3 4233
Depth 4 4233
Depth 5 4233
Depth 6 4233
Depth 7 4233
Depth 8 4233
Depth 9 4233
Depth 10 4233
Depth 11 4233
Depth 12 93581
Depth 13 580567
Depth 14 2439924
Depth 15 6623373
Depth 16 13680851
Depth 17 20285657
Depth 18 22777813
Depth 19 23004075
Depth 20 23148005
Depth 21 23252552
Depth 22 23306255
Depth 23 23338776
Depth 24 23355650
Depth 25 23359586
MAX ('LT', 'EV', 1933, ('KS', 'HO', 'TJ', 'HS', 'JH', 'DV', 'EV', 'EG', 'WI', 'FT'))
1933
